In [1]:
import pandas as pd

In [3]:
#Leo el csv de eventos

events_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [4]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Generación de features
---

In [7]:
features_df = pd.DataFrame()

In [8]:
features_df['person'] = events_df.person.unique()

In [9]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad eventos totales

In [10]:
df = events_df.groupby('person').event.count()

In [11]:
features_df = features_df.join(df, on='person')

In [12]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

In [13]:
features_df.head()

,person,cant_eventos_totales
0,4886f805,9
1,ad93850f,65
2,0297fc1e,567
3,2d681dd8,26
4,cccea85e,836


## Cantidad por evento

In [14]:
df = events_df.groupby('person').event.value_counts()

In [17]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [16]:
features_df = features_df.join(df, on='person')

In [19]:
features_df.rename(columns={'ad campign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

In [20]:
features_df.head()

,person,cant_eventos_totales,ad campaign hit,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs
0,4886f805,9,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,4.0,1.0
1,ad93850f,65,10.0,15.0,1.0,NaN,7.0,NaN,7.0,NaN,NaN,20.0,5.0
2,0297fc1e,567,29.0,4.0,7.0,NaN,21.0,1.0,NaN,6.0,NaN,404.0,95.0
3,2d681dd8,26,1.0,5.0,1.0,NaN,1.0,NaN,2.0,1.0,NaN,13.0,2.0
4,cccea85e,836,15.0,7.0,1.0,NaN,20.0,NaN,26.0,1.0,5.0,739.0,22.0


In [21]:
features_df.fillna(0, inplace=True)

In [22]:
features_df.head()

,person,cant_eventos_totales,ad campaign hit,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs
0,4886f805,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0
1,ad93850f,65,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0
2,0297fc1e,567,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0
3,2d681dd8,26,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0
4,cccea85e,836,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0


## Cantidad de returnings

In [23]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [24]:
features_df = features_df.join(df, on='person')

In [25]:
features_df.head()

,person,cant_eventos_totales,ad campaign hit,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,Returning
0,4886f805,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,NaN
1,ad93850f,65,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0,4.0
2,0297fc1e,567,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0,94.0
3,2d681dd8,26,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0,1.0
4,cccea85e,836,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0,21.0


## Tiempo total

In [35]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [36]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

person
0008ed71    0 days 04:00:50
00091926   27 days 21:43:34
00091a7a    0 days 00:05:47
000ba417    9 days 01:57:37
000c79fe    0 days 00:10:20
Name: timestamp, dtype: timedelta64[ns]

In [ ]:
features_df = features_df.join(df, on='person')

In [43]:
features_df.rename(columns={'timestamp'})

0     0 days 00:23:08
1     7 days 23:00:32
2   138 days 12:53:38
3     9 days 19:34:20
4    23 days 16:05:31
Name: timestamp, dtype: timedelta64[ns]

# Predicciones
---

In [32]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Separación de datos

In [27]:
labels_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [28]:
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 15)

In [29]:
labels = train_df.label
train_df = train_df.drop(columns=['label'])

In [30]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 14)

In [ ]:
data_dmatrix = xgb.DMatrix(data=train_df.drop(columns=['person']) , label=labels)

In [ ]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

## Modelo

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [ ]:
xg_reg.fit(X_train,y_train)

## Tests

In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
roc_auc_score(y_test,preds)

## Predicciones posta

In [ ]:
preds_posta = xg_reg.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_posta

## Generación del csv a publicar

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person
to_publish.shape

In [ ]:
to_publish['label'] = preds_posta
to_publish.shape

In [ ]:
to_publish.head()

In [ ]:
to_publish.to_csv('27_11_2.csv', index=False)

# Algo de cross validation
---

In [ ]:
params = {'objective':'binary:logistic', 'base_score':'0.3', 
                      'colsample_bylevel':'0.7', 'booster':'dart', 
                      'colsample_bytree':'0.4', 'learning_rate':'0.25',
                      'max_depth':'13', 'alpha':'25', 'gamma':'10', 
                      'n_estimators':'23', 'eval_metric':'auc'}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123)

In [ ]:
cv_results.head()

In [ ]:
import matplotlib as plt

xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()